In [ ]:
#all imports
!pip install wikipedia
import wikipedia
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('all')

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=d21aefd6a346586ce174c5ccca78751174ecc36dea2145a58fb1eefb50d2e48a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
#Word wrap
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#Remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#extracting the text from wikipedia via the wikipedia api
def getText():
  #checks if the topic is valid, if not it will prompt the user again
  while True:
    try:
      topic = input("Enter a topic: ")
      text = wikipedia.summary(topic)
    except Exception:
      print("Not Found")
    #returns the data and the name of the topic
    else:
      return text, topic

In [ ]:
def preprocess(text):
  #tokenizes the sentence
  word_tokens = nltk.word_tokenize(text.lower())
  #removes stop words
  filtered_words = [w for w in word_tokens if not w in stopwords.words('english')]
  #tags the words with pos tags
  pos_tags = nltk.pos_tag(word_tokens)
  #the lemmatizer used
  lemmatizer = WordNetLemmatizer()

  #iterates through the text and extracts the nouns and verbs only
  word_lemmas = []
  for token, pos_tag in zip(word_tokens, pos_tags):
    if pos_tag[1][0] in ['N', 'V', 'A', 'R']:
      lemma = lemmatizer.lemmatize(token)
      word_lemmas.append(lemma)

  return word_lemmas

In [ ]:
def response(text, question):
  #creates and array of sentences and adds the question to the end
  sentence_tokens = nltk.sent_tokenize(text)
  sentence_tokens.append(question)

  #creates the vectorizer
  tfidfVector = TfidfVectorizer(tokenizer=preprocess)

  #transforms the array of sentences into vectors
  tfidf = tfidfVector.fit_transform(sentence_tokens)

  #calculates the cosine_similarity between the question and text
  values = cosine_similarity(tfidf[-1], tfidf)

  #assigns the index of the sentence with the highest similarity to index
  index = values.argsort()[0][-2]

  #flattens the values
  values_flat = values.flatten()
  values_flat.sort()

  #if the value of the most similar sentence is greater than 0 then return the sentence assigned to that value
  coeff = values_flat[-2]
  if coeff > 0 :
    return sentence_tokens[index]
  #otherwise return "i dont understand"
  else:
    return "I don't understand."

In [ ]:
#chatloop
data, topic = getText()
while True:
  question = input("What would you like to know about " + topic + "?\n")
  #if user enters quit then exit loop else answer prompt
  if(question.lower() == "quit"):
    break;
  #else answer the question
  else:
    output = response(data, question)
    print("-" + output + "\n")
